In [1]:
from os import listdir
from os.path import isfile, join
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.tokenize import word_tokenize
import pickle
from gensim.similarities import WmdSimilarity
from gensim.summarization.summarizer import summarize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ronnald/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
with open('label_content', 'rb') as f:
    labelContent = pickle.load(f)

In [4]:
tagged_data = [TaggedDocument(words=word_tokenize(content.lower()), tags=[str(label)]) for label, content in labelContent.items()]

In [5]:
max_epochs = 10
vec_size = 100#neurons
alpha = 0.025
window = 3

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1,
                window = window,binary=True)
  
model.build_vocab(tagged_data)



In [6]:
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    #model.min_alpha = model.alpha

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [7]:
to_measure = {}
for i in range(0,len(tagged_data)):
    to_measure[i] =tagged_data[i][1][0]

In [24]:
from time import time
import numpy as np
start = time()

Jict = {}
distance =np.zeros(shape=(len(to_measure),len(to_measure)))

for i in range(0,len(to_measure)):
    Jict[to_measure[i]] = {}
    for j in range(0,len(to_measure)):
        rel = model.wv.wmdistance(document1=to_measure[i],document2=to_measure[j])
        Jict[to_measure[i]][to_measure[j]] = {'weight': rel}
        distance[i][j] = rel

print('Took %.2f seconds to run.' % (time() - start))

Took 0.21 seconds to run.


In [32]:
#Took 426.52 seconds to run.

In [48]:
print(to_measure[482])
print(to_measure[395])

2017-2-257741
2017-2-257756


In [75]:
print(tagged_data[11][1][0])
print(tagged_data[357][1])

2018-1-302067
['2017-1-182232']


In [12]:
with open('label_sentence', 'rb') as f:
    labelSentence = pickle.load(f)

In [85]:
print(summarize(labelSentence['2017-1-182232'],ratio=0.1))


O cenario de conflitos religiosos e de intolerancia faz ainda menos sentido quando nos deparamos com informacoes, dados e estatisticas acerca da diversidade religiosa cada vez maior no Brasil e no mundo.
Ao mesmo tempo em que observamos a crescente diversidade e pluralismo religioso no mundo, nos deparamos com o fenomeno de intolerancia religiosa.


In [86]:
print(summarize(labelSentence["2017-2-257741"],ratio=0.1))


O cenario de conflitos religiosos e de intolerancia faz ainda menos sentido quando nos deparamos com informacoes, dados e estatisticas acerca da diversidade religiosa cada vez maior no Brasil e no mundo.
Contudo, inversamente, ao mesmo tempo em que observamos a crescente diversidade e pluralismo religioso no mundo, nos deparamos com o fenomeno de intolerancia religiosa.


In [20]:
with open('wmd', 'wb') as f:
        pickle.dump(distance, f)


In [33]:
with open('Jict', 'wb') as f:
        pickle.dump(Jict, f)